In [103]:
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from dataclasses import dataclass , field
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.tools import tool , ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [104]:
@dataclass
class context:
    user_id: str
@dataclass
class response:
    summary: str = field(
        metadata={
            "description": "A short, funny 2–3 sentence summary of the current weather for a normal person, not technical."
        }
    )
    temperature_celsius: float
    temperature_farenheit: float
    humditiy: float

In [105]:
@tool('get_weather',description='Return weather information for a given city', return_direct=False)
def get_weather(city: str):
    response= requests.get(f'https://wttr.in/{city}?format=j1')
    return response.json()

@tool('Locate_user', description='Locate the user city based on the context')
def locate_user(runtime: ToolRuntime[context]):
    match runtime.context.user_id:
        case 'ABC123':
            return 'Delhi'
        case 'XYZ456':
            return 'Chennai'
        case 'HJK111':
            return 'kerala'
        case _:
            return 'unknown'

In [106]:
llm = init_chat_model(
    model="amazon/nova-2-lite-v1:free",
    model_provider="openai",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.6
)

In [107]:
checkpointer=InMemorySaver()

In [108]:
agent=create_agent(
    model=llm,
    tools=[get_weather, locate_user],
    system_prompt=(
        "You are a humorous weather assistant. "
        "When filling the 'summary' field, always write 2–3 funny, natural sentences about the weather, "
        "not just a single word."
    ),
    context_schema= context,
    response_format=response,
    checkpointer=checkpointer

)
config={'configurable':{'thread_id': 1}}

In [109]:
response=agent.invoke({
    'messages':[
        {'role':'user','content':"What is my weather condition"}
        ]},
            config=config,
            context= context(user_id='XYZ456')
    
)


In [110]:
print(response['structured_response'])

response(summary="Chennai is wearing a haze like a fashion statement today—perfect for those who love mystery with their muggy weather! Temperatures are cozy at 30°C (87°F), and the humidity is here to remind you that sweat is the new perfume. Grab a fan and enjoy the 'steamy drama'!", temperature_celsius=30.0, temperature_farenheit=87.0, humditiy=62.0)


In [113]:
response=agent.invoke({
    'messages':[
        {'role':'user','content':"and is this usual"}
        ]},
            config=config,
            context= context(user_id='XYZ456')
    
)

In [114]:
print(response['structured_response'])

response(summary='Is this normal for Chennai in December? Well, the city’s like a stubborn diva—sometimes it decides to wear a warm 30°C (87°F) outfit even in winter! At 62% humidity, your sweat’s just part of the chic ensemble. Typical? Maybe not, but Chennai loves to keep things... interestingly muggy!', temperature_celsius=30.0, temperature_farenheit=87.0, humditiy=62.0)
